This notebook estimates BLP demand side model by optimising GMM objective function to generate demand side estimates. Refer here for theoretical underpinnings!

In [ ]:
# =======================================
# Import Required Libraries
# =======================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
import pickle
import multiprocessing as mp
from scipy import stats, special
from scipy.optimize import minimize
from scipy.special import logsumexp
from sklearn.linear_model import LinearRegression
from numba import jit, njit, prange
import pyblp
import importlib
import sys
import pdb

# Optional installs (if not already installed)
# !pip install miceforest
# !pip install pyblp
# !pip install linearmodels


# Display full DataFrame when inspecting
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# =======================================
# File Paths (Update if needed)
# =======================================
phone_path = "/content/Phonearena_data(Feb,2024).csv"
cpi_path = "/content/CPI data ( 2009-2023).csv"
final_data_path = "/content/FINAL DATA_1.csv"
function_path = "/content/myfunctions_Dec_31_24.py"


In [ ]:
# =======================================
# 🔧 3. Import Custom Functions
# =======================================
sys.path.append('/content')
from myfunctions_Dec_31_24 import *
import myfunctions_Dec_31_24
importlib.reload(myfunctions_Dec_31_24)


In [ ]:
# =======================================
#  Load Main Dataset
# =======================================

df = pd.read_csv(final_data_path)
df.drop(columns=['Unnamed: 0'], inplace=True)

# =======================================
#  Basic Cleaning and Sorting
# =======================================
df.sort_values(by='Quarter', inplace=True)

# Remove duplicate rows based on product characteristics within a quarter
df.drop_duplicates(
    subset=["Quarter", "Brand", "Screen Size", "Storage (GB)", "RAM (GB)", "Model Name"],
    keep='first',
    inplace=True

In [ ]:
# =======================================
# Generate Unique Identifiers
# =======================================

# Assign a model ID to every row (for models that may repeat with different specs)
df.insert(9, 'Model_Id', [f'ID_{i+1}' for i in range(len(df))])

# Factorize brand name to assign unique brand IDs
df['Brand_Id'] = pd.factorize(df['Brand'])[0]

# Optional: Dictionary to look up brand name from brand ID
brand_id_lookup = dict(enumerate(df['Brand'].unique()))

# Define key characteristics (to be used later in analysis)
characteristics_columns = ['Screen Size', 'RAM (GB)', 'weight(g)', 'Processor Speed Band', 'Price']


In [ ]:

# =======================================
# Extract Weight from Text
# =======================================
def extract_grams(weight_string):
    """Extract numerical gram value from string."""
    if isinstance(weight_string, str):
        match = re.search(r"(\d+\.\d+)\s*g", weight_string)
        if match:
            return float(match.group(1))
    return np.nan

df['weight(g)'] = df['Weight:'].apply(extract_grams)



# =======================================
#  Fill Missing Weights Manually
# =======================================
# Weights manually collected from reliable sources. 

smartphone_weights_grams = {
    "SAMSUNGGALAXYS":116,
    'MOTOROLABRAVO': 122.0,
    'T-MOBILEMYTOUCH': 138.0,
    'T-MOBILECOMET': 115.0,
    'MOTOROLADROIDX': 155.0,
    'LGVORTEX': 139.0,
    'LGOPTIMUSM': 128.0,
    'LGOPTIMUSS': 130.0,
    'HUAWEIASCEND': 130.0,
    'HTCHD2': 157.0,
    'HTCEVO': 170.0,
    'HTCINSPIRE4G': 164.0,
    'MOTOROLADROIDX2': 158.0,
    'SAMSUNGDROIDCHARGE': 142.88,
    'LGOPTIMUX2X': 147.0,
    'HUAWEIM835': 110.0,
    'HTCEVO3D': 170.0,
    'AT&TIMPULSE': 125.0,
    'LGTHRILL': 168.0,
    'ZTESCORE': 125.0,
    'SAMSUNGGALAXYS2SKYROCKET': 132.0,
    'SAMSUNGGALAXYNEXUS': 135.0,
    'SAMSUNGFOCUSFLASH': 116.2,
    'HTCHEROS': 120.0,
    'SAMSUNGGALAXYATTAIN': 132.0,
    'SAMSUNGGALAXYSBLAZE': 128.0,
    'SAMSUNGFOCUS2': 122.0,
    'SAMSUNGGALAXYSAVIATOR': 144.0,
    'LGLUCID': 127.0,
    'MOTOROLADROIDRAZRM': 126.0,
    'T-MOBILEMYTOUCHQ': 134.0,
    'SAMSUNGGALAXYSLIGHTRAY': 142.0,
    'SAMSUNGREVERB': 127.5,
    'LGSPLENDO': 147.0,
    'KYOCERARISE': 130.0,
    'SAMSUNGGALAXYY': 97.5,
    'ZTEANTHEM': 138.0,
    'LGSPECTRUM2': 145.0,
    'HUAWEIASCENDY': 120.0,
    'HUAWEIFUSION2': 120.0,
    'HUAWEIASCENDP1': 130.0,
    'SAMSUNGATIVODYSSEY': 125.0,
    'LGSPIRIT': 118.0,
    'PANTECHPERCEPTION': 140.0,
    'ZTEVITAL': 150.0,
    'ZTEIMPERIAL': 159.0,
    'LGOPTIMUSGPRO': 172.0,
    'SAMSUNGGALAXYDISCOVER': 122.0,
    'ZTEOPEN': 117.8,
    'LGOPTIMUSF6': 122.0,
    'ALCATELONETOUCHSCRIBE': 135.0,
    'LGOPTIMUSEXTREME': 130.0,
    'HUAWEIVITRIA': 146.0,
    'ZTESOURCE': 136.5,
    'ALCATELONETOUCHEVOLVE': 136.5,
    'LGOPTIMUSDYNAMIC': 124.7,
    'LGOPTIMUSEXCEED2': 135.0,
    'ZTEVALET': 136.5,
    'COOLPADFLO': 140.0,
    'LGGVISTA': 173.0,
    'MOTOROLAMOTOE': 142.0,
    'LGTRIBUTE': 139.0,
    'ZTEWARPSYNC': 145.0,
    'COOLPADQUATTROII': 141.5,
    'LGF60': 127.0,
    'ZTESPEED': 155.0,
    'ZTEGRANDXMAXPLUS': 170.0,
    'LGLEON': 140.0,
    'BLUSTUDIO6.0HD': 190.0,
    'ALCATELONETOUCHPOPSTAR2': 147.0,
    'ASUSZENFONE2': 272.0, # This weight is as per previous data, consider if it's correct for a phone.
    'HUAWEISNAPTO': 140.0,
    'BLUSTUDIO5.0CHD': 160.0,
    'ZTESOLAR': 155.0,
    'COOLPADROGUE': 130.0,
    'HTCDESIRE626S': 140.0,
    'BLUVIVOSELFIE': 150.0,
    'LGGVISTA2': 165.0,
    'ALCATELONETOUCHFIERCEXL': 173.0,
    'BLUDASHX': 148.0,
    'BLULIFEONEX(2016)': 141.0,
    'BLUSTUDIOGPLUS': 170.0,
    'LGTRIBUTE5': 143.0,
    'BLUVIVOXL': 152.0,
    'BLUSTUDIOGHD': 130.0,
    'BLUDASHXPLUS': 190.0,
    'SAMSUNGGALAXYJ3V': 152.0,
    'BLUDASHL': 122.0,
    'ALCATELONETOUCHPIXIECLIPSE': 295.0, # As noted before, this seems like a tablet weight.
    'BLUDASHM': 157.0,
    'BLUNEOXPLUS': 227.0, # This still seems high for a phone, but kept from source.
    'BLUADVANCE5.0': 140.6,
    'COOLPADCATALYST': 139.0,
    'MOTOROLAMOTOZ': 136.0,
    'ALCATELONETOUCHPIXI4(5)': 168.0,
    'BLUR1HD': 144.0,
    'ALCATELONETOUCHPOP4PLUS': 156.0,
    'HUAWEISENSA': 155.0,
    'NOKIA6': 169.0,
    'SAMSUNGGALAXYJ3PRIME': 148.0,
    'ZTEMAJESTYPRO': 140.0,
    'ZTEBLADEXMAX': 180.0,
    'ZTEMAXXL': 176.0,
    'ZTEBLADEMAX3': 176.0,
    'SAMSUNGGALAXYJ7PERX': 167.0,
    'SAMSUNGGALAXYS8ACTIVE': 208.0,
    'HTCDESIRE555': 160.0,
    'NOKIA2': 153.0,
    'ZTEAVID4': 160.0,
    'ZTETEMPOGO': 140.0,
    'HONOR7X': 165.0,
    'LGSTYLO4': 172.0,
    'LGK30': 160.0,
    'SAMSUNGGALAXYJ3(2018)': 152.0,
    'BLUDASHL5': 148.0,
    'LGK8(2018)': 152.0,
    'MOTOROLAMOTOE5': 174.0,
    'ALCATELTETRA': 150.0,
    'BLUVIVOX': 165.0,
    'SAMSUNGGALAXYSOL3': 133.0,
    'ZTEVISIBLER2': 150.0,
    'BLUPUREVIEW': 155.0,
    'BLUC5': 146.0,
    'AT&TAXIA': 748.0, # This is still likely the package weight. Specific phone weight not found.
    'COOLPADLEGACY': 180.0,
    'LGESCAPEPLUS': 170.0,
    'COOLPADLEGACYS': 180.0,
    'LGNEONPLUS': 170.0,
    'LGSTYLO6': 219.0,
    'LGXPRESSIONPLUS3': 152.0,
    'ONEPLUS8T': 188.0,
    'TCLA3': 160.0,
    'ZTEBLADE11PRIME': 186.0,
    'TCL20S': 190.0
}


# Apply mapping for models with missing weights
df['weight(g)'] = df['weight(g)'].fillna(df['Model Name'].map(smartphone_weights_grams))



In [ ]:

# =======================================
# CPI Adjustment for Real Prices
# =======================================
cpi = pd.read_csv(cpi_path)

# Merge CPI by year into main dataset
df = pd.merge(df, cpi, on="Year", how='left')

# Normalize CPI to 2011 base
cpi_base = df.loc[df['Year'] == 2011, 'CPI'].values[0]
df['CPI_index'] = df['CPI'] / cpi_base

# Create inflation-adjusted price columns
df['Adj_prices'] = df['ASP'] * df['CPI_index']
df['Price'] = df['Adj_prices'] / 1000  # Price scaled in $1000s



# =======================================
# Normalize Exchange Rates
# =======================================
normalising_exchange(df, ['Japan', 'Korea', 'China'])

# =======================================
# Merge PhoneArena Dataset (RAM, Camera, Processor)
# =======================================
phone_arena = pd.read_csv(phone_path)[['Model Name', 'RAM:', 'Processor:', 'Main camera:']]
merged = pd.merge(df, phone_arena, on='Model Name', how='left')

# =======================================
# Fill Missing Features (RAM, Camera, Processor)
# =======================================
# RAM
df['Ram'] = df['RAM (GB)'].fillna(df['RAM:']).fillna(merged['RAM:_y'])
df['Ram'] = df['Ram'].astype(str).str.extract(r'(\d+\.\d+|\d+)').astype(float)

# Camera
df['Camera'] = df['Main camera:'].fillna(df['\n\nCamera\n\n']).fillna(merged['Main camera:_y'])
df['Camera'] = df['Camera'].astype(str).str.extract(r'(\d+\.\d+|\d+)').astype(float)

# Fix known errors
df.loc[df['Model Name'].isin(['BLACKBERRY8820', 'BLACKBERRY8830']), 'Camera'] = 0
df.loc[df['Model Name'] == "MOTOROLAMOTOE(2015)", 'Model Name'] = "MOTOROLAMOTOE(2NDGEN)"
df.loc[df['Model Name'] == "SAMSUNGGALAXYNEXUS", "Camera"] = 5



# =======================================
# Extract Processor Speed in GHz
# =======================================
# Fix band values like "08: 2.8GHz+" → "3GHz"
df['Processor Speed Band'] = df['Processor Speed Band'].replace({'08: 2.8GHz+': '3GHz'})

# Extract GHz from Processor Speed Band
df['Processor Speed (Band)'] = df['Processor Speed Band'].str.extract(r'([\d\.]+)').astype(float)

# Extract MHz from Processor description
mhz = df['Processor:'].str.split(',', expand=True)[1]
df['Processor Speed (Raw)'] = mhz.str.extract(r'([\d\.]+)').astype(float) / 1000

# Fill processor speed from raw, then fallback to band
df['Processor Speed'] = df['Processor Speed (Raw)'].fillna(df['Processor Speed (Band)'])

# =======================================
#  Compute smartphone Model Age
# =======================================
df['Quarter_q'] = df['Quarter'].copy()
df['Quarter'] = pd.to_datetime(df['Quarter'])

# First appearance of each model
first_seen = df.groupby('Model Name')['Quarter'].min()
df['first_appeared'] = df['Model Name'].map(first_seen)
df['first_appeared'] = pd.to_datetime(df['first_appeared'])

# Age in quarters
df['Age'] = ((df['Quarter'].dt.year - df['first_appeared'].dt.year) * 4 +
             (df['Quarter'].dt.quarter - df['first_appeared'].dt.quarter))


In [ ]:


# =======================================
#  Log-Transforms and Scaling
# =======================================
df['Storage (GB)'] = df['Storage (GB)'] / 1000
df['Screen Size'] = df['Screen Size'] / 10
df['Ram'] = df['Ram'] / 10
df['Camera'] = df['Camera'] / 100
df['Processor Speed'] = df['Processor Speed'] / 10
df['weight(g)'] = df['weight(g)'] / 100

# Recompute log characteristics
char_cols = ['Screen Size', 'Camera', 'Processor Speed', 'weight(g)', 'Price']
ln_cols = [f'ln_{col}' for col in char_cols]
df[ln_cols + ['ln_Age', 'ln_Spec_tech']] = np.log(df[char_cols + ['Age', 'Spec_tech']])



In [ ]:

# =======================================
# Filter for Estimation Sample
# =======================================
# Remove units with low sales
df = df[df['Units'] > 5000]

# Drop rows with missing/zero values in key characteristics
key_cols = ["Screen Size", "Storage (GB)", "Camera", "Processor Speed", "Age", "Spec_tech"]
df = df.dropna(subset=key_cols)
df = df[(df["Storage (GB)"] > 0) & (df["Camera"] > 0) & (df["Processor Speed"] > 0) & (df["Screen Size"] > 0)]

# Filter out pre-2011
df = df[df['Year'] != 2010]



In [ ]:


# =======================================
# Variables for GMM Estimation
# =======================================
df['Market size'] = 200_000_000
df['P_share'] = df['Units'] / df['Market size']
df['Outside_share'] = 1 - df.groupby('Quarter')['P_share'].transform("sum")
df['diff2'] = np.log(df['P_share']) - np.log(df['Outside_share'])
df['ln_price'] = np.log(df["Price"])
df['Cons'] = 1

# Time trend (used for Spec_tech parameter)
quarter_map = {q: i+1 for i, q in enumerate(df['Quarter'].unique())}
df['Time Trend'] = df['Quarter'].map(quarter_map)

# =======================================
# Instrument Construction & Dropping
# =======================================
instrument_cols = generate_instruments(df, ["Cons", "Screen Size", "Storage (GB)", 'Camera', "Processor Speed", "weight(g)"])
df = df.dropna(subset=instrument_cols + ['Japan', 'Korea', 'China'])
df = df.loc[~((df['Camera'] == 0) | (df['Storage (GB)'] == 0))]





# =======================================
# Save Clean Version 
# =======================================
df_original = df.copy()

In [ ]:
# Select final columns to retain
final_cols = [
    'Quarter', 'Quarter_q', 'Year', 'Brand', 'Model Name', 'Model_Id', 'Brand_Id', 'OS Version',
    'Screen Size', 'Storage (GB)', 'weight(g)', 'Ram', 'Processor Speed', 'Age', 'Camera',
    'Spec_tech', 'Price', 'Units', 'ln_Screen Size', 'ln_Camera', 'ln_Processor Speed', 'ln_Price',
    'ln_Age', 'ln_Spec_tech', 'Japan', 'Korea', 'China', 'Japan_n', 'Korea_n', 'China_n',
    'first_appeared', 'ln_weight(g)'
]
df = df[final_cols]

#Constructing Dummies#


In [ ]:

# =======================================
# Dummies
# =======================================

#storage dummies

storage_bins = [0, 1, 4, 16, 64, 128, 256, 512, 1024, 2048, 5120]
df['storage_bin'] = pd.cut(df['Storage (GB)'], bins=storage_bins, labels=False)
storage_dummies = pd.get_dummies(df['storage_bin'], prefix="Storage", dtype=float, drop_first=True)

# Mahalanobis and Isolation Forest outliers
df['Outlier'] = get_isolation_forest_outliers(df, characteristics_columns_p)
df['mahal'] = get_mahalanobis_distances(df, characteristics_columns_p)

# Thresholding
threshold = np.sqrt(chi2.ppf(0.997, df=5))  # 5 variables included
df = df[(df['Outlier'] == 1) & (df['mahal'] <= threshold)]




In [ ]:
# =======================================
# --- DUMMY VARIABLES ---
# =======================================

specific_brands = ['Apple', 'LG', 'Motorola', 'Samsung']
df['Brand_d'] = df['Brand'].apply(lambda x: x if x in specific_brands else 'Other')
brand_dummies = pd.get_dummies(df['Brand_d'], dtype=int).drop('Other', axis=1)

tech_dummies = pd.get_dummies(df['Spec_tech'], dtype=int)
tech_dummies = tech_dummies.drop([2.0, 2.5, 3.0], axis=1)

df['Model Name_d'] = df['Model Name'].apply(lambda x: x if df.loc[df['Model Name'] == x, 'Brand'].values[0] in specific_brands else 'Other')
model_dummies = pd.get_dummies(df['Model Name_d'], dtype=int).drop('Other', axis=1)

to_drop_q = ["2017-01-01", "2017-04-01", "2017-07-01", "2017-10-01"]
quarter_dummies = pd.get_dummies(df['Quarter'], dtype=int).drop(to_drop_q, axis=1)

time_trend_m = df['Time Trend'].to_numpy().reshape(-1, 1)
tech_technology_m = tech_dummies.to_numpy() * time_trend_m

df['Brand_quarter'] = df['Brand_d'] + '-' + df['Quarter'].astype(str)
brand_quarter_dummies = pd.get_dummies(df['Brand_quarter'], dtype=int)
brand_quarter_dummies = brand_quarter_dummies.loc[:, ~brand_quarter_dummies.columns.str.contains('Other')]
brand_quarter_dummies = brand_quarter_dummies.drop(['Apple-2021-04-01', 'LG-2021-04-01', 'Motorola-2014-04-01', 'Samsung-2011-01-01'], axis=1)

df['Brand_Year'] = df['Brand_d'] + '-' + df['Year'].astype(str)
brand_year_dummies = pd.get_dummies(df['Brand_Year'], dtype=int).drop('Samsung-2020', axis=1)

year_dummies = pd.get_dummies(df['Year'], drop_first=True, dtype=int)

In [ ]:

# =======================================
#  INTERACTIONS & FINAL DATA MATRICES ---
# =======================================


df['Age_square'] = df['Age'] ** 2
delta_0 = df['diff2'].values
J = df.groupby('Quarter')['Model Name'].count().values
T = len(J)
N = 500  # Simulated individuals per market




# =======================================
	# Scale characteristics
# =======================================

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[characteristics_columns_p + ['Age']])
df_scaled = df.copy()
df_scaled[characteristics_columns_p + ['Age']] = X_scaled
df_scaled['Cons'] = 1

X_q = df[q_cols].to_numpy()
X_rand_p = df_scaled[["Screen Size", "Camera", 'weight(g)', "Cons", "Price"]]
X_rand = df_scaled[["Screen Size", "Camera", 'weight(g)', "Cons"]]
X_rand_q = df_scaled[["Price", "Cons"]]

X_d = df_scaled[["Cons", "Screen Size", "Camera", 'weight(g)', "Age"]]
X_d = pd.concat([X_d, tech_dummies, quarter_dummies, brand_dummies, brand_quarter_dummies], axis=1)

X_d_p = df_scaled[["Cons", "Screen Size", "Camera", 'weight(g)', "Age", "Price"]]
X_d_p = pd.concat([X_d_p, tech_dummies, quarter_dummies, brand_dummies, brand_quarter_dummies], axis=1)

X_d_m = X_d.to_numpy()
X_d_p_m = X_d_p.to_numpy()


In [ ]:


# =======================================
# 		Instrument matrix
# =======================================

inst_cols = ["Screen Size", "Storage (GB)", "weight(g)", 'Camera', 'Processor Speed']
exch_cols = ['Japan_n', 'Korea_n', 'China_n']

df_pyblp_inst = df[inst_cols + ['Quarter_q', "Brand_Id"]]
rename_dict = {'Screen Size': 'Screen', 'Storage (GB)': "Storage", 'Camera': 'Camera', 'Processor Speed': 'Processor',
               'Brand_Id': 'firm_ids', "Quarter_q": "market_ids", "weight(g)": "weight"}
formulation = 'Screen + Storage + Camera + weight + Processor'

pyblp_inst = prepare_pyblp_instruments(df_pyblp_inst, inst_cols, rename_dict, formulation)[1]
pyblp_inst_diff = prepare_pyblp_instruments(df_pyblp_inst, inst_cols, rename_dict, formulation)[2]

Z_p = df[instr_cols + exch_cols]
Z_p_m = Z_p.to_numpy()
exch_inst_m = df[exch_cols].to_numpy()
Z_fan_yang = fan_yang_inst(df_scaled, inst_cols).to_numpy()
Z_fan_yang_1 = fan_yang_inst_1(df_scaled, inst_cols)


In [ ]:

# =======================================
# 		VIF check and drop
# =======================================

vif_Z = compute_vif(Z_fan_yang_1)
high_vif_cols = vif_Z[vif_Z['VIF'] > 10]['Variable'].tolist()
Z_fan_yang_1 = Z_fan_yang_1.drop(high_vif_cols, axis=1)
Z_fan_yang_1_m = Z_fan_yang_1.to_numpy()

# IV matrix setup
X_iv = pd.concat([df[["Screen Size", "Camera", 'weight(g)', "Age"]], tech_dummies, quarter_dummies, brand_dummies, brand_quarter_dummies], axis=1)
X_iv_m = X_iv.to_numpy()


# =======================================
#     Final instrument sets
# =======================================

Z_non_pyblp = np.hstack((X_iv_m, Z_p_m))
Z_pyblp = np.hstack((X_iv_m, pyblp_inst))
Z_pyblp_diff = np.hstack((X_iv_m, pyblp_inst_diff))
Z_fan_yang_m = np.hstack((X_iv_m, Z_fan_yang))
Z_fan_yang_1_m = np.hstack((X_iv_m, Z_fan_yang_1_m))

Z_pyblp_ex = np.hstack((Z_pyblp, exch_inst_m))
Z_pyblp_diff_ex = np.hstack((Z_pyblp_diff, exch_inst_m))
Z_fan_yang_ex = np.hstack((Z_fan_yang_m, exch_inst_m))
Z_fan_yang_1_ex = np.hstack((Z_fan_yang_1_m, exch_inst_m))

# =======================================
#  		Instrument selector
# =======================================

inst_map = {
    0: Z_pyblp,
    1: Z_pyblp_diff,
    2: Z_fan_yang_m,
    3: Z_fan_yang_1_m,
    4: Z_non_pyblp,
    5: Z_pyblp_ex,
    6: Z_pyblp_diff_ex,
    7: Z_fan_yang_ex,
    8: Z_fan_yang_1_ex
}
inst_type = 3
Z = inst_map.get(inst_type)

In [ ]:

# =======================================
# 			Diagnostics
# =======================================

cond_dict = {}
for key, matrix in inst_map.items():
    cond_dict[key] = {
        "full_rank": np.linalg.matrix_rank(matrix) == matrix.shape[1],
        "condition_number": np.linalg.cond(matrix)
    }
# =======================================
# 	QR decomposition for multicollinearity analysis
# =======================================

Q, R, piv = qr(Z_fan_yang_1_m, mode='economic', pivoting=True)
independent_cols = sorted(piv[:np.linalg.matrix_rank(Z_fan_yang_1_m)])
dependent_cols = sorted(set(range(Z_fan_yang_1_m.shape[1])) - set(independent_cols))

Z_df = pd.concat([X_iv, Z_fan_yang_1], axis=1)
independent_names = Z_df.columns[independent_cols]
dependent_names = Z_df.columns[dependent_cols]



#Some Preliminary analyses/Results#

In [ ]:

# --- Descriptive stats and summaries ---
columns_describe = ['Units'] + characteristics_columns
describe = df.groupby(['Brand', 'Quarter', "Model Name"])[columns_describe].sum().reset_index()
describe['Units'] /= 1000
describe = describe.sort_values('Quarter')

# ---------------------------------------
# OLS Regression: Price ~ Instruments
# ---------------------------------------
print("-" * 60)
print("OLS regression of Price on Instruments (Z_fan_yang_1)")
print("-" * 60)

# Fit OLS model
ols_price_iv = sm.OLS(p, Z_fan_yang_1).fit()

# Print adjusted R-squared
print("Adjusted R-squared:", ols_price_iv.rsquared_adj)

# Print full summary
print(ols_price_iv.summary())




quarterly_results = describe.groupby('Quarter')[columns_describe].mean().reset_index()
stats = quarterly_results.describe().loc[['mean', 'min', 'max', 'std']].transpose()
print(stats)
print(describe.shape)

# --- OLS Regressions ---
df['const'] = 1
x = pd.concat([df[characteristics_columns + ['Age', 'Price', 'Time Trend']], tech_dummies, Brand_dummies], axis=1)
full_model_ols = sm.OLS(df['diff2'], x).fit()

print("OLS with Trend, Tech, Brand Dummies")
print(summary_col([full_model_ols], stars=True))

# --- OLS with interaction terms ---
interaction_cols = create_interaction_columns(df, tech_dummies=tech_dummies,
                                              brand_dummies=Brand_dummies,
                                              tech_levels=[4.0, 4.5, 5.0],
                                              brands=specific_brands)
x_int = pd.concat([
    df[characteristics_columns + ['Age', 'Price', 'Time Trend'] + interaction_cols],
    tech_dummies,
    Brand_dummies
], axis=1)
full_model_int_ols = sm.OLS(df['diff2'], x_int).fit()
print(full_model_int_ols.summary())

# --- 2SLS regressions ---
y = df['diff2']
x = pd.concat([X_iv, X_d_p[['Age', 'Price']], Year_dummies], axis=1)
Z_ls = pd.concat([x, Z_fan_yang], axis=1)
result = two_stage_using_api(y, x, Z_ls)
print("2SLS Coefficients:", result.params)
print("2SLS Std Errors:", result.bse)
print(result.summary())

# --- 2SLS using IV2SLS ---
X_endog = X_d_p['Price']
W_exog = pd.concat([Brand_dummies, tech_dummies, Year_dummies, df[characteristics_columns + ['Age']]], axis=1)
Z_instr = Z_fan_yang_1
iv_model = IV2SLS(dependent=y, exog=W_exog, endog=X_endog, instruments=Z_instr).fit()
print(iv_model.summary)
print("Sargan stat:", iv_model.sargan)

# --- 2SLS with interaction terms ---
W_exog_int = pd.concat([
    Brand_dummies,
    tech_dummies,
    df[characteristics_columns + ['Age', 'Time Trend'] + interaction_cols]
], axis=1)
iv_model_int = IV2SLS(y, W_exog_int, endog=X_endog, instruments=Z_instr).fit(cov_type='kernel')
print(iv_model_int.summary)

# --- IV Logit First Stage Diagnostics ---
X_tilde = np.hstack((X_d_m, p.reshape(-1, 1)))
zxw1 = Z.T @ X_d_p_m

if np.linalg.matrix_rank(Z) < Z.shape[1]:
    raise ValueError("Multicollinearity in instrument matrix Z")
if np.linalg.matrix_rank(X_d_p_m) < X_d_p_m.shape[1]:
    raise ValueError("Multicollinearity in characteristics matrix X_d_p_m")

bx1 = np.linalg.inv(zxw1.T @ zxw1) @ zxw1.T @ Z.T @ delta_0
e = delta_0 - (X_d_p_m @ bx1)
g_ind = e.reshape((-1, 1)) * Z
demean = g_ind - g_ind.mean(axis=0).reshape((1, -1))
vg = demean.T @ demean / demean.shape[0]
w0 = np.linalg.inv(vg)
cc_1 = np.linalg.inv(zxw1.T @ w0 @ zxw1) @ zxw1.T @ w0 @ Z.T @ delta_0

print("Initial weighting matrix stats:", w0.max(), w0.min())
print(cc_1)

In [ ]:

# --- Initial Guess and GMM Objective ---
sigma_v = np.random.normal(0, 1, size=k)
t0 = time.time()
delta_guess = df.diff2.values

if not np.allclose(delta_guess, d.delta):
    raise ValueError("Delta guess mismatch")

obj = objective_1(param=sigma_v,
                  shares=df.P_share.values,
                  X_rand=X_rand_m,
                  X_d=X_d_m,
                  v_it=v_it,
                  prices=p,
                  J=J, T=T, N=N,
                  tol=1e-3,
                  delta=delta_guess,
                  Z=Z,
                  W=w0)
print("GMM Objective Initial Value:", obj)
print("Computation Time:", time.time() - t0)



In [ ]:

# ---------------------------
# Sanity Checks and Quick Diagnostics
# ---------------------------

print("X_iv shape:", X_iv.shape)
print("Z_fan_yang_1 shape:", Z_fan_yang_1.shape)
print("Correlation matrix of Z:")
display(pd.DataFrame(Z).corr())
print("Eigenvalues of Z.T @ Z:")
print(np.linalg.eigvals(Z.T @ Z))
print("Initial delta_0:", delta_0)

# Inspect key matrices
print("Head of X_rand:")
display(X_rand.head())
print("Summary of X_rand:")
print(X_rand.describe())
print("Models with Camera >= 5.3:")
display(df[df_scaled['Camera'] >= 5.3])


In [ ]:

# ---------------------------
# Indirect Utility Computation (Validation Run)
# ---------------------------

_ = compute_indirect_utility(X_rand_p_m, v_it, p, df.diff2.values, sigma_v, J, T, N)


In [ ]:
# ---------------------------
# Reload Custom Functions
# ---------------------------

import importlib
import myfunctions_Dec_31_24
importlib.reload(myfunctions_Dec_31_24)


# ---------------------------
# Optimization Setup
# ---------------------------

max_time = 60 * 60   # Max optimization time (1 hour)
inst_type = 8        # Fan & Yang 1 instruments + exchange rates
n_1_1 = 19           # Number of linear parameters to track

# Run the GMM optimization
logger = run_optimization(
    X_rand_m=X_rand_m,
    X_d_p=X_d_p,
    X_d_p_m=X_d_p_m,
    v_it=v_it,
    p=p,
    J=J,
    T=T,
    N=N,
    Z=Z,
    delta_guess=df.diff2.values,
    df=df,
    max_optimisation_time=max_time,
    inst_type=inst_type,
    k=k
)

In [ ]:

# ---------------------------
# Best Results Summary
# ---------------------------

print("\n=== Best GMM Solution Found ===")
print(f"Objective value: {logger.best_objective}")

print("\nNon-linear parameters (σ_v):")
for name, value in zip(logger.param_names, logger.best_params['sigma_v']):
    print(f"{name}: {value:.6f}")

print("\nLinear parameters (β):")
print(logger.best_params['linear_params'])

print("\nStandard errors:")
print(logger.best_params['std_errors'])

# Get long format parameter DataFrame
param_df = logger.get_linear_param_long_format()

# Inspect price coefficient over iterations
display(param_df[param_df['parameter'] == "Price"])

# Timestamp of completion
from datetime import datetime
from zoneinfo import ZoneInfo
print("Eastern Time:", datetime.now(ZoneInfo("America/New_York")))

# Get iteration with minimum objective value
min_obj_dict = min(logger.iteration_data, key=lambda x: x['objective_value'])
min_iter = min_obj_dict['iteration']

best_df = param_df[param_df['iteration'] == min_iter]
print("Best parameter estimates from iteration", min_iter)
display(best_df)

print("Best σ_v:", min_obj_dict['sigma_v'])
print("Std errors from best iteration:", min_obj_dict['std_errors'])


In [ ]:
# ---------------------------
# GMM Moments and Weight Matrix Recalculation
# ---------------------------

def gmm_moments(sigma_v_3):
    delta_tmp = solve_delta_1(df.P_share.values, X_rand_m, v_it, p, delta_check, sigma_v_check, J, T, N, 1e-5)
    xi_tmp = delta_tmp - X_d_p_m @ b_check
    moment_tmp = xi_tmp.reshape((-1, 1)) * Z
    return moment_tmp

b_check = logger.best_params['linear_params']
sigma_v_check = logger.best_params['sigma_v']
delta_check = logger.best_params['delta']
obs_check = Z.shape[0]

# Recompute moments and weight matrix
xi_check = delta_check - X_d_p_m @ b_check
g_check = xi_check.reshape((-1, 1)) * Z
vg_check = (g_check.T @ g_check) / obs_check
weight_check = np.linalg.inv(vg_check)

# GMM robust standard errors
_ = gmm_moments(sigma_v_check)
Gmm_std_error(sigma_v_check, b_check, gmm_moments, xi_check, Z, X_d_p_m, weight_check)

# Final export
param_df.to_csv("/content/param_df(06_23_2025).csv", index=False)